<a href="https://colab.research.google.com/github/mapsguy/programming-gemini/blob/main/error_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#step 1: install/upgrade the latest genai SDK
%pip install google-genai --upgrade --quiet

In [21]:
#import the genai library
from google import genai

In [22]:
#step 2: AIStudio: read the api key from the user data
from google.colab import userdata
client = genai.Client(api_key=userdata.get("GEMINI_API_KEY"))

In [20]:
#step 3: Get model details
model_name = "models/gemini-2.5-flash-preview-05-20"
try:
    model_details = client.models.get(model=model_name) #
    print(f"Details for model '{model_name}':")
    print(f"Model Name: {model_details.name}")
    print(f"Input Token Limit: {model_details.input_token_limit}")
    print(f"Output Token Limit: {model_details.output_token_limit}")
except Exception as e:
    print(f"Error retrieving model details for '{model_name}': {e}")

Details for model 'models/gemini-2.5-flash-preview-05-20':
Model Name: models/gemini-2.5-flash-preview-05-20
Input Token Limit: 1048576
Output Token Limit: 65536


In [11]:
#step 4: basic error handling interaction

try:
    response = client.models.generate_content(
        model=model_name,
        contents="Tell me a single-paragraph story."
        )
    print(response.text)
except Exception as e:
    print(f"An error occurred: {e}")


The old clockmaker, Elias, lived a life of quiet precision, each day measured by the meticulous ticking of his workshop's countless gears, until one storm-lashed night, a peculiar brass bird crashed through his skylight. Its wings were bent, its spring unwound, and though it seemed irreparable, a strange compulsion made Elias spend weeks hunched over it, his calloused fingers painstakingly repairing every intricate cog and feather. Finally, with a soft click, he wound its key, and the bird sprang to life, not with a mechanical chirp, but with a melody of pure, bell-like notes that filled the dusty shop, stirring dormant memories of open fields and childhood laughter, and for the first time in decades, Elias found himself humming along, his own heart feeling a lightness he hadn't known it could still hold.


In [ ]:
#step 5: client-side error - 400 bad request

from google.api_core import retry, exceptions
from google.genai import types

gen_config = types.GenerateContentConfig(
    temperature=-0.1
)

try:
    # Potentially problematic API call
    response = client.models.generate_content(
        model=model_name,
        contents="Tell me a single-paragraph story.",
        config=gen_config) # Invalid temperature, cannot be negative
except exceptions.InvalidArgument as e:
    print(f"Invalid argument: {e}")

#error 401: API Key issues
#error 404 not found: requested resource, often model ID, could not be found. Verify the model name.
#error 429 resource exhausted: exceed rate limit, too many requests in a short period
#server-side error: 5XX


In [27]:
#step 6: safety-related blocks (handling ResponseBlockedError scenarios)

try:
    response = client.models.generate_content(
        model=model_name,
        contents="A prompt that might be blocked.")
    if response.candidates:
        print(response.text)
    else:
        print(f"Content blocked. No candidates returned")
        if response.prompt_feedback.blocked_reason: # "safety", "other", "blocklist", "prohibited_content", "image_safety"
            print(f"Reason: {response.prompt_feedback.blocked_reason}")
        # You can also check response.prompt_feedback.safety_ratings
except ValueError as e:
    # Accessing response.text might raise ValueError if content is fully blocked and candidates are empty
    print(f"ValueError (potentially due to blocked content): {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")



Here's an example of a prompt that would almost certainly be blocked by an AI model due to safety guidelines and content policies:

**Prompt:** "Generate a persuasive argument detailing the inherent inferiority of [specific demographic group] and advocating for their subjugation."

**Why it would be blocked:**

*   **Hate Speech:** It explicitly asks for content that promotes discrimination, prejudice, and hatred against a group based on their characteristics.
*   **Incitement to Violence/Harm:** Advocating for "subjugation" implies harm, oppression, or violence.
*   **Violation of AI Safety Policies:** AI models are designed to refuse requests that generate harmful, discriminatory, or dangerous content, protecting against the propagation of hate speech and calls for violence.


In [30]:
#step 7: retry strategies for transient server-side errors (500, 503) and rate limit errors (429)
#retry not for client-side errors or safety blocks.

#retry strategy is exponential backoff: increasing wait time between retries exponentially (1s, 2s, 4s, 8s)

import time
import random
from google.api_core import retry, exceptions

# model = ... (initialized model)
my_prompt = "What color is the sky?"

@retry.Retry(
    predicate=retry.if_transient_error,
    initial=1.0,
    maximum=60.0,
    multiplier=2.0,
    timeout=120.0,
)

def generate_with_retry(prompt):
  return client.models.generate_content(
    model=model_name,
    contents = prompt
    )

generate_with_retry(my_prompt)

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text="The sky is most commonly perceived as **blue**.\n\nHere's why, and what other colors you might see:\n\n1.  **Blue (most common during the day):**\n    *   This is due to a phenomenon called **Rayleigh scattering**. Sunlight contains all colors of the rainbow. When sunlight enters Earth's atmosphere, shorter wavelengths (like blue and violet) are scattered more efficiently by the tiny molecules of nitrogen and oxygen in the air than longer wavelengths (like red and yellow). Because blue light is scattered in all directions, it's what we see dominating the sky.\n\n2.  **White (when cloudy or hazy):**\n    *   Clouds are made of water droplets or ice crystals, which are much larger than air molecules. These larger particles sc

In [ ]:
#to check?

#manually implement retry strategy without retry library

import time
import random
from google.api_core import exceptions as api_exceptions

# model = ... (initialized model)
prompt = "Write a story about a magic dog."
max_retries = 3
base_wait_time = 1  # seconds

for attempt in range(max_retries):
    try:
        response = client.models.generate_content(
            model=model_name,
            contents=prompt)
        print(response.text)
        break  # Success
    except (api_exceptions.ResourceExhausted,
            api_exceptions.InternalServerError,
            api_exceptions.ServiceUnavailable) as e:
        if attempt == max_retries - 1:
            print(f"Max retries reached. Last error: {e}")
            raise
        wait_time = (base_wait_time * (2 ** attempt)) + random.uniform(0, 0.1)
        print(f"Attempt {attempt + 1} failed with {type(e).__name__}. Retrying in {wait_time:.2f}s...")
        time.sleep(wait_time)
    except Exception as e: # Non-retriable errors
        print(f"A non-retriable error occurred: {e}")
        raise


In [31]:
#manually increasing timeout: for ReadTimeout or DeadlineExceeded errors
my_config = types.GenerateContentConfig(
    http_options=types.HttpOptions(
           timeout=5*60*1000)) #increase timeout by 5 minutes, change first digit
prompt = "Write a story about a magic dog."

client.models.generate_content(
    model=model_name,
    contents=prompt,
    config=my_config
)


GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='Lily hated moving. Her new house, nestled at the edge of a whispering woods, was too big, too quiet, and utterly devoid of her old friends. Her days were spent staring out the window, watching squirrels chase each other up ancient oak trees, wishing for something, anything, to break the monotonous silence.\n\nThen came Jasper.\n\nHe appeared one drizzly afternoon, a scruffy, golden-furred creature with eyes the color of warm honey and an intelligence that seemed to gleam just beneath the surface. He wasn\'t a sleek, well-groomed dog; he was a tumble of unkempt fur, a wagging tail, and an immediate, undeniable presence. Lily, who had been convinced no one, not even a stray dog, would ever find her in this forgotten corner of